# Mixed Integer Programming(MIP) for the instance  'Meal Planning for the New Millennium'(MnM)

## Download the library 
First install [DOcplex](https://cdn.rawgit.com/IBMDecisionOptimization/docplex-doc/2.0.15/docs/index.html) Python library if needed. Consider the scale for this problem, [CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [DOcplexcloud](https://developer.ibm.com/docloud) is needed. 

In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Set up the prescriptive engine

Subscribe to DOcplexcloudor Decision Optimization on Cloud solve service here if you do not want to use a local solver.
Get the service URL and your personal API key and enter your credentials here if accurate:

In [ ]:
# url = None
# key = None

## Create a model instance

In [2]:
from docplex.mp.model import Model

MnM = Model(name='Meal Planning for the New Millennium')

The model for this instance is describe below
<p>
<ul>
<img src = "https://i.imgur.com/xB0qG3N.png" width="50%" height="50%" >
<img src = "https://i.imgur.com/loWaatP.png" >
</ul> 


## Define the variables

In [3]:
# x, y, z are binary variable
x = {(i): MnM.binary_var(name='x_{0}'.format(i)) for i in range(21)}
y = {(i,j): MnM.binary_var(name='y_{0}_{1}'.format(i,j)) for i in range(21) for j in range(5)}
z = {(i,j): MnM.binary_var(name='z_{0}_{1}'.format(i,j)) for i in range(21) for j in range(5)}

In [4]:
# for limit the size of nutrient, cooking time, expence
fplus = {(i): MnM.continuous_var(name='fplus_{0}'.format(i), lb=0) for i in range(4)}
fminus = {(i): MnM.continuous_var(name='fminus_{0}'.format(i), lb=0) for i in range(4)}
eplus = MnM.continuous_var(name='eplus', lb=0)
eminus = MnM.continuous_var(name='eminus', lb=0)
w = MnM.continuous_var(name='w', lb=0)

## Define the parameters

In [5]:
# recipe matrix 
rating = [4.38, 4.38, 4.38, 5.0, 4.38, 4.38, 3.8, 4.38, 4.38, 3.13, 3.75, 4.38, 4.38, 4.38, 3.75, 3.13, 3.75, 4.38, 5.0 , 4.38, 3.75]
calories = [1172, 298, 682, 517, 856, 599 , 67, 746, 354, 352 ,  658 ,  426 ,  414 ,  745, 689 ,  352 ,  250 ,  463 ,  347 ,  333 ,  695]
protein = [ 54 ,   6  ,   36  ,    7  ,   45  ,   48  ,    2 ,   118  ,   16 ,   16 ,   7 ,   9,   5 ,   41,  85 ,   6 ,  33 ,   5 ,   3 ,  12 ,   29]
fat = [73  ,  12  ,   57 ,  18  , 54  ,  28 ,   4 ,  21 ,   22 ,  31 ,  39 ,   28 ,   37 ,   60, 34 ,   4 ,  11 ,  21 ,    7 ,   26 ,   45]
sodium = [220  ,  199  ,  909 ,   20  , 1797 , 1038 ,    66, 483, 930 ,  488 ,  209 ,   408,   310 ,  1146, 723 , 139 ,  123 , 150 ,   89 ,   693 ,  1251]
price = [9.38, 11.97 , 7.26 ,4.97 , 11.75 , 7.99 , 5.01,14.87 ,10.28 , 4.38, 9.38 ,  8.14 ,  5.32, 11.37,10.23, 7.64 , 9.45 , 6.76, 4.27 ,12.87 ,3.99]
p_times = [120,30 , 60 ,42, 84, 54,36,150,102, 30,78,90,30,120,90, 60,108,90,30,120 ,60]
remaintime_y =[70,50,100,100,120]
remaintime_z=[150,120,100,80,80]

# lower(1) and upper(2) bound of nutrient
c_bound=[2700, 3600]
p_bound= [ 140 ,200]
f_bound=[150 , 250]
s_bound= [2400 ,3000]
e_bound= 25

# parameter for objective
beta = [1 ,1 , 1 , 1]
gamma = 0.1
alpha = 0.1

## Define the constraints

In [6]:
# five meals constrain
MnM.add_constraint(MnM.sum(x[i] for i in range(21)) - 5 == 0, ctname = 'subject to five_meals_total')
MnM.add_constraint(MnM.sum(y) <= 5)
MnM.add_constraint(MnM.sum(z) <= 5)

# nutrition lower bound
MnM.add_constraint(MnM.sum(calories[i]*x[i] for i in range(21)) >= c_bound[0])
MnM.add_constraint(MnM.sum(protein[i]*x[i] for i in range(21)) >= p_bound[0])
MnM.add_constraint(MnM.sum(fat[i]*x[i] for i in range(21)) >= f_bound[0])
MnM.add_constraint(MnM.sum(sodium[i]*x[i] for i in range(21)) >= s_bound[0])

# nutrition/price upper bound
MnM.add_constraint(MnM.sum(calories[i]*x[i] for i in range(21)) - c_bound[1] == fplus[0] - fminus[0])
MnM.add_constraint(MnM.sum(protein[i]*x[i] for i in range(21)) - p_bound[1] == fplus[1] - fminus[1])
MnM.add_constraint(MnM.sum(fat[i]*x[i] for i in range(21)) - f_bound[1] == fplus[2] - fminus[2])
MnM.add_constraint(MnM.sum(sodium[i]*x[i] for i in range(21)) - s_bound[1] == fplus[3] - fminus[3])
MnM.add_constraint(MnM.sum(price[i]*x[i] for i in range(21)) - e_bound == eplus - eminus)

# number of cooking constraint
MnM.add_constraint(MnM.sum(y) + MnM.sum(z) - MnM.sum(x) == 0)

        
# schedule time constraint
MnM.add_constraint(MnM.sum(p_times[i]* (y[i,j] - z[i,j]) for i in range(21) for j in range(5)) <= w)
MnM.add_constraint(MnM.sum(p_times[i]* (z[i,j] - y[i,j]) for i in range(21) for j in range(5)) <= w)

        
# schedule date constraint
for j in range(5):
    MnM.add_constraint(MnM.sum(y[i,j] for i in range(21)) <= 1)
    MnM.add_constraint(MnM.sum(z[i,j] for i in range(21)) <= 1)
    
# cooking time constraint
for j in range(5):
    MnM.add_constraint(MnM.sum(p_times[i]*y[i,j] for i in range(21)) <= remaintime_y[j])
    MnM.add_constraint(MnM.sum(p_times[i]*z[i,j] for i in range(21)) <= remaintime_z[j])  
    
# not repeat constraint
for i in range(21):
    MnM.add_constraint(MnM.sum(y[i,j] + z[i,j] for j in range(5)) <= 1)
    
# assignment constraints
for i in range(21):
    MnM.add_constraint(MnM.sum(y[i,j] + z[i,j] for j in range(5)) == x[i])


## Define the objective function

In [7]:
MnM.maximize(MnM.sum(rating[i]*x[i] for i in range(21)) - MnM.sum(beta[i]*fplus[i] for i in range(4)) - gamma*eplus*eplus - alpha*w)

## Solve the problem

In [8]:
MnM.print_information()

Model: Meal Planning for the New Millennium
 - number of variables: 242
   - binary=231, integer=0, continuous=11
 - number of constraints: 77
   - linear=77
 - parameters: defaults


In [9]:
MnMs= MnM.solve(log_output=True)
assert MnMs
MnM.print_solution()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIQP Presolve eliminated 33 rows and 71 columns.
MIQP Presolve modified 285 coefficients.
Aggregator did 1 substitutions.
Reduced MIQP has 43 rows, 170 columns, and 946 nonzeros.
Reduced MIQP has 164 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 1 nonzeros.
Presolve time = 0.01 sec. (1.74 ticks)
Probing time = 0.00 sec. (0.41 ticks)
Tried aggregator 1 time.
Reduced MIQP has 43 rows, 170 columns, and 946 nonzeros.
Reduced MIQP has 164 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 1 nonzeros.
Presolve time = 0.00 sec. (0.48 ticks)
Probing time = 0.00 sec. (0.41 ticks)
Clique table members: 30.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.01 sec. (1.34 ticks)

        Nodes                                         